In [119]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import myslack_incomming
import scipy
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingRegressor 

## Preprocessing Step2 -  Encoding

In [2]:
myslack_incomming.send_slack('Encoding started.')

<Response [200]>


In [3]:
train = pickle.load(open("./data/imputation_train.pkl", "rb"))
test = pickle.load(open("./data/imputation_test.pkl", "rb"))

In [4]:
train.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
647049,39,191346,Sunday,3.239000e+10,1,PHARMACY OTC,1118.0
647050,39,191346,Sunday,7.874205e+09,1,FROZEN FOODS,1752.0
647051,39,191346,Sunday,4.072000e+03,1,PRODUCE,4170.0
647052,8,191347,Sunday,4.190008e+09,1,DAIRY,1512.0
647053,8,191347,Sunday,3.800060e+09,1,GROCERY DRY GOODS,3600.0


---

In [5]:
train.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
647049,39,191346,Sunday,3.239000e+10,1,PHARMACY OTC,1118.0
647050,39,191346,Sunday,7.874205e+09,1,FROZEN FOODS,1752.0
647051,39,191346,Sunday,4.072000e+03,1,PRODUCE,4170.0
647052,8,191347,Sunday,4.190008e+09,1,DAIRY,1512.0
647053,8,191347,Sunday,3.800060e+09,1,GROCERY DRY GOODS,3600.0


In [6]:
train[train['VisitNumber']==8].groupby('DepartmentDescription')['ScanCount'].sum()

DepartmentDescription
DAIRY                        2
DSD GROCERY                  1
HOUSEHOLD CHEMICALS/SUPP     1
MEAT - FRESH & FROZEN        1
PAINT AND ACCESSORIES       19
PETS AND SUPPLIES            4
Name: ScanCount, dtype: int64

# 파생변수 생성

### Train

In [82]:
train['Count'] = train['ScanCount']

In [83]:
df = train.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['Count'].sum()
df1 = df.groupby(['VisitNumber'], as_index=False)['Count'].min()
df2 = df.groupby(['VisitNumber'], as_index=False)['Count'].max()
df3 = df.groupby(['VisitNumber'], as_index=False)['Count'].mean()
df1.rename(columns={'Count': 'Min'}, inplace=True)
df2.rename(columns={'Count': 'Max'}, inplace=True)
df3.rename(columns={'Count': 'Mean'}, inplace=True)
train = train.merge(df1, how='left', on=['VisitNumber'], copy=True)
train = train.merge(df2, how='left', on=['VisitNumber'], copy=True)
train = train.merge(df3, how='left', on=['VisitNumber'], copy=True)         
train['Max - Min'] = train['Max'] - train['Min'] 

In [84]:
df = train.groupby(['VisitNumber',  'FinelineNumber'], as_index=False)['Count'].count()
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.rename(columns={'Count': 'FLN_count'}, inplace=True) 
train = train.merge(df, how='left', on=['VisitNumber'], copy=True)

In [85]:
df = train.groupby(['VisitNumber',  'Upc'], as_index=False)['Count'].count()
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.rename(columns={'Count': 'Upc_count'}, inplace=True) 
train = train.merge(df, how='left', on=['VisitNumber'], copy=True) 

In [86]:
df = train.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['Count'].count()
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.rename(columns={'Count': 'DD_count'}, inplace=True) 
train = train.merge(df, how='left', on=['VisitNumber'], copy=True)

In [87]:
df = train.groupby(['VisitNumber'], as_index=False)['Count'].sum()
df.rename(columns={'Count': 'Sum'}, inplace=True) 
train = train.merge(df, how='left', on=['VisitNumber'], copy=True)

In [88]:
train['FLN_DD_Ratio'] = train['FLN_count']/train['DD_count']
train['UPc_DD_Ratio'] = train['Upc_count']/train['DD_count'] 
train['mean_to_min'] = train['Mean']/train['Min']
train['mean_to_min'][train['Min']==0] = 0
train['max_to_mean'] = train['Max']/train['Mean']
train['max_to_mean'][train['Mean']==0] = 0  

In [89]:
train.drop(columns='Count',inplace=True)

In [101]:
train.tail(10)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Max - Min,FLN_count,Upc_count,DD_count,Sum,FLN_DD_Ratio,UPc_DD_Ratio,mean_to_min,max_to_mean
647044,39,191346,Sunday,5.100020e+09,1,GROCERY DRY GOODS,3107.0,1,5,2.125,4,16,17,8,17,2.0,2.125,2.125,2.352941
647045,39,191346,Sunday,7.874204e+09,1,DAIRY,7955.0,1,5,2.125,4,16,17,8,17,2.0,2.125,2.125,2.352941
647046,39,191346,Sunday,3.120020e+09,1,DSD GROCERY,4639.0,1,5,2.125,4,16,17,8,17,2.0,2.125,2.125,2.352941
647047,39,191346,Sunday,3.120033e+09,1,DSD GROCERY,4639.0,1,5,2.125,4,16,17,8,17,2.0,2.125,2.125,2.352941
647048,39,191346,Sunday,3.700091e+09,1,HOUSEHOLD CHEMICALS/SUPP,8947.0,1,5,2.125,4,16,17,8,17,2.0,2.125,2.125,2.352941
647049,39,191346,Sunday,3.239000e+10,1,PHARMACY OTC,1118.0,1,5,2.125,4,16,17,8,17,2.0,2.125,2.125,2.352941
647050,39,191346,Sunday,7.874205e+09,1,FROZEN FOODS,1752.0,1,5,2.125,4,16,17,8,17,2.0,2.125,2.125,2.352941
647051,39,191346,Sunday,4.072000e+03,1,PRODUCE,4170.0,1,5,2.125,4,16,17,8,17,2.0,2.125,2.125,2.352941
647052,8,191347,Sunday,4.190008e+09,1,DAIRY,1512.0,1,1,1.000,0,2,2,2,2,1.0,1.000,1.000,1.000000
647053,8,191347,Sunday,3.800060e+09,1,GROCERY DRY GOODS,3600.0,1,1,1.000,0,2,2,2,2,1.0,1.000,1.000,1.000000


### test

In [91]:
test['Count'] = test['ScanCount']

In [92]:
df = test.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['Count'].sum()
df1 = df.groupby(['VisitNumber'], as_index=False)['Count'].min()
df2 = df.groupby(['VisitNumber'], as_index=False)['Count'].max()
df3 = df.groupby(['VisitNumber'], as_index=False)['Count'].mean()
df1.rename(columns={'Count': 'Min'}, inplace=True)
df2.rename(columns={'Count': 'Max'}, inplace=True)
df3.rename(columns={'Count': 'Mean'}, inplace=True)
test = test.merge(df1, how='left', on=['VisitNumber'], copy=True)
test = test.merge(df2, how='left', on=['VisitNumber'], copy=True)
test = test.merge(df3, how='left', on=['VisitNumber'], copy=True)         
test['Max - Min'] = test['Max'] - test['Min'] 

In [93]:
df = test.groupby(['VisitNumber',  'FinelineNumber'], as_index=False)['Count'].count()
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.rename(columns={'Count': 'FLN_count'}, inplace=True) 
test = test.merge(df, how='left', on=['VisitNumber'], copy=True)

In [94]:
df = test.groupby(['VisitNumber',  'Upc'], as_index=False)['Count'].count()
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.rename(columns={'Count': 'Upc_count'}, inplace=True) 
test = test.merge(df, how='left', on=['VisitNumber'], copy=True) 

In [95]:
df = test.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['Count'].count()
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.rename(columns={'Count': 'DD_count'}, inplace=True) 
test = test.merge(df, how='left', on=['VisitNumber'], copy=True)

In [96]:
df = test.groupby(['VisitNumber'], as_index=False)['Count'].sum()
df.rename(columns={'Count': 'Sum'}, inplace=True) 
test = test.merge(df, how='left', on=['VisitNumber'], copy=True)

In [97]:
test['FLN_DD_Ratio'] = test['FLN_count']/test['DD_count']
test['UPc_DD_Ratio'] = test['Upc_count']/test['DD_count'] 
test['mean_to_min'] = test['Mean']/test['Min']
test['mean_to_min'][test['Min']==0] = 0
test['max_to_mean'] = test['Max']/test['Mean']
test['max_to_mean'][test['Mean']==0] = 0  

In [98]:
test.drop(columns='Count',inplace=True)

In [100]:
test.tail(10)

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Max - Min,FLN_count,Upc_count,DD_count,Sum,FLN_DD_Ratio,UPc_DD_Ratio,mean_to_min,max_to_mean
653636,191341,Sunday,8.861669e+10,1,INFANT APPAREL,6405.0,1,5,2.400000,4,10,10,5,12,2.000000,2.000000,2.400000,2.083333
653637,191341,Sunday,7.352824e+10,2,INFANT CONSUMABLE HARDLINES,1009.0,1,5,2.400000,4,10,10,5,12,2.000000,2.000000,2.400000,2.083333
653638,191341,Sunday,8.861669e+10,1,INFANT APPAREL,6407.0,1,5,2.400000,4,10,10,5,12,2.000000,2.000000,2.400000,2.083333
653639,191348,Sunday,8.046920e+10,1,SWIMWEAR/OUTERWEAR,321.0,2,3,2.333333,1,7,7,3,7,2.333333,2.333333,1.166667,1.285714
653640,191348,Sunday,8.874396e+10,1,MENS WEAR,1857.0,2,3,2.333333,1,7,7,3,7,2.333333,2.333333,1.166667,1.285714
653641,191348,Sunday,6.657211e+10,1,BATH AND SHOWER,1505.0,2,3,2.333333,1,7,7,3,7,2.333333,2.333333,1.166667,1.285714
653642,191348,Sunday,8.818139e+10,1,BATH AND SHOWER,1099.0,2,3,2.333333,1,7,7,3,7,2.333333,2.333333,1.166667,1.285714
653643,191348,Sunday,4.282557e+09,1,MENS WEAR,8220.0,2,3,2.333333,1,7,7,3,7,2.333333,2.333333,1.166667,1.285714
653644,191348,Sunday,8.046919e+10,1,SWIMWEAR/OUTERWEAR,114.0,2,3,2.333333,1,7,7,3,7,2.333333,2.333333,1.166667,1.285714
653645,191348,Sunday,7.871536e+09,1,MENS WEAR,4923.0,2,3,2.333333,1,7,7,3,7,2.333333,2.333333,1.166667,1.285714


### 2-0-1. Train TripType

In [102]:
samsub = pd.read_csv('./data/sample_submission.csv')

In [103]:
samsub.tail()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
95669,191338,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95670,191339,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95671,191340,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95672,191341,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95673,191348,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
train_TT_dummy_df = pd.get_dummies(train['TripType'], prefix='TripType_')

In [105]:
train_TT_dummy_df.tail()

,TripType__3,TripType__4,TripType__5,TripType__6,TripType__7,TripType__8,TripType__9,TripType__12,TripType__14,TripType__15,...,TripType__36,TripType__37,TripType__38,TripType__39,TripType__40,TripType__41,TripType__42,TripType__43,TripType__44,TripType__999
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
647052,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

### 2-1-1. Train Weekday

In [106]:
# before
train['Weekday'].value_counts()

Sunday       133975
Saturday     122096
Friday        96247
Monday        83130
Tuesday       72529
Wednesday     71115
Thursday      67962
Name: Weekday, dtype: int64

In [107]:
WD_columns = train['Weekday'].value_counts().index
WD_columns

Index(['Sunday', 'Saturday', 'Friday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'],
      dtype='object')

In [108]:
# after
def weekday_preprocessing(dataframe):
    weekday_dict={
        'Sunday':1,
        'Saturday':2,
        'Friday':3,
        'Monday':4,
        'Tuesday':5,
        'Wednesday':6,
        'Thursday':7,
    }
    return dataframe['Weekday'].map(weekday_dict)

train['Weekday'] = weekday_preprocessing(train)
train['Weekday'].value_counts()

1    133975
2    122096
3     96247
4     83130
5     72529
6     71115
7     67962
Name: Weekday, dtype: int64

In [109]:
train_WD_dummy_df = pd.get_dummies(train['Weekday'])
train_WD_dummy_df.columns = WD_columns

In [110]:
train_WD_dummy_df.tail()

,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday
647049,1,0,0,0,0,0,0
647050,1,0,0,0,0,0,0
647051,1,0,0,0,0,0,0
647052,1,0,0,0,0,0,0
647053,1,0,0,0,0,0,0


### 2-1-2. Test Weekday

In [111]:
# before
test['Weekday'].value_counts()

Sunday       133233
Saturday     123974
Friday        94764
Monday        85174
Tuesday       73742
Wednesday     72481
Thursday      70278
Name: Weekday, dtype: int64

In [112]:
WD_columns = test['Weekday'].value_counts().index
WD_columns

Index(['Sunday', 'Saturday', 'Friday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'],
      dtype='object')

In [113]:
# after
def weekday_preprocessing(dataframe):
    weekday_dict={
        'Sunday':1,
        'Saturday':2,
        'Friday':3,
        'Monday':4,
        'Tuesday':5,
        'Wednesday':6,
        'Thursday':7,
    }
    return dataframe['Weekday'].map(weekday_dict)

test['Weekday'] = weekday_preprocessing(test)
test['Weekday'].value_counts()

1    133233
2    123974
3     94764
4     85174
5     73742
6     72481
7     70278
Name: Weekday, dtype: int64

In [114]:
test_WD_dummy_df = pd.get_dummies(test['Weekday'])
test_WD_dummy_df.columns = WD_columns

In [115]:
test_WD_dummy_df.tail()

,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday
653641,1,0,0,0,0,0,0
653642,1,0,0,0,0,0,0
653643,1,0,0,0,0,0,0
653644,1,0,0,0,0,0,0
653645,1,0,0,0,0,0,0


---

### 2-2. Upc Skip

---

### 2-3-1. Train DepartmentDescription

In [116]:
print(len(train['DepartmentDescription'].sort_values().unique()))
train['DepartmentDescription'].sort_values().unique()

67


array(['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY',
       'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES',
       'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES',
       'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD',
       'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY',
       'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES',
       'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14',
       'GROCERY DRY GOODS', 'HARDWARE', 'HEALTH AND BEAUTY AIDS',
       'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS',
       'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS',
       'IMPULSE MERCHANDISE', 'INFANT APPAREL',
       'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES',
       'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS',
       'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN',
       'MEDIA AND GAMING', 'MENS WEAR', 'OFFICE SUPPLIES',
       'OPTICAL - FRAMES', 'OPTICAL - LENSES', '

In [117]:
train_DD_dummy_df = pd.get_dummies(train['DepartmentDescription'])
train_DD_dummy_df.tail()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
train_DD_dummy_df_columns=[]
for idx in preprocessing.LabelEncoder().fit_transform(train_DD_dummy_df.columns):
    train_DD_dummy_df_columns.append('DD_{}'.format(idx))
train_DD_dummy_df.columns = train_DD_dummy_df_columns
train_DD_dummy_df.tail()

,DD_0,DD_1,DD_2,DD_3,DD_4,DD_5,DD_6,DD_7,DD_8,DD_9,...,DD_57,DD_58,DD_59,DD_60,DD_61,DD_62,DD_63,DD_64,DD_65,DD_66
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2-3-2. Test DepartmentDescription

In [121]:
print(len(test['DepartmentDescription'].sort_values().unique()))
test['DepartmentDescription'].sort_values().unique()

66


array(['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY',
       'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES',
       'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES',
       'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD',
       'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY',
       'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES',
       'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14',
       'GROCERY DRY GOODS', 'HARDWARE', 'HOME DECOR', 'HOME MANAGEMENT',
       'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP',
       'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL',
       'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES',
       'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS',
       'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN',
       'MEDIA AND GAMING', 'MENS WEAR', 'OFFICE SUPPLIES',
       'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS',
       'PAINT

'HEALTH AND BEAUTY AIDS' DD가 test에는 없다.!!!

In [122]:
test_DD_dummy_df = pd.get_dummies(test['DepartmentDescription'])
test_DD_temp_dummy_df = pd.concat([test_DD_dummy_df.loc[:,test_DD_dummy_df.columns[:26]], 
                                   pd.DataFrame([0]*len(test), columns=['HEALTH AND BEAUTY AIDS'])], axis=1)
test_DD_dummy_df = pd.concat([test_DD_temp_dummy_df,
                              test_DD_dummy_df.loc[:,test_DD_dummy_df.columns[26:]]],axis=1)
test_DD_dummy_df.tail()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
653641,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
test_DD_dummy_df_columns=[]
for idx in preprocessing.LabelEncoder().fit_transform(test_DD_dummy_df.columns):
    test_DD_dummy_df_columns.append('DD_{}'.format(idx))
test_DD_dummy_df.columns = test_DD_dummy_df_columns
test_DD_dummy_df.tail()

,DD_0,DD_1,DD_2,DD_3,DD_4,DD_5,DD_6,DD_7,DD_8,DD_9,...,DD_57,DD_58,DD_59,DD_60,DD_61,DD_62,DD_63,DD_64,DD_65,DD_66
653641,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2-4-1. Train FinelineNumber

In [124]:
len(train['FinelineNumber'].sort_values().unique())

5195

In [125]:
%%time
# in Test, not in Train
test_unique_fln = test['FinelineNumber'].sort_values().unique()
train_unique_fln = train['FinelineNumber'].sort_values().unique()

in_test_not_in_train = [i for i in test_unique_fln if i not in train_unique_fln]

CPU times: user 220 ms, sys: 12.9 ms, total: 233 ms
Wall time: 232 ms


In [126]:
len(in_test_not_in_train)

158

In [127]:
train_FLN_dummy_df = pd.get_dummies(train['FinelineNumber'], prefix='FLN_')
train_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN__9964.0,FLN__9966.0,FLN__9967.0,FLN__9970.0,FLN__9971.0,FLN__9974.0,FLN__9975.0,FLN__9991.0,FLN__9997.0,FLN__9998.0
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
FLN_zero_series = pd.Series([0]*len(train))
train_FLN_zero_dummy_df = pd.concat([FLN_zero_series, FLN_zero_series],axis=1)
for _ in tqdm(range(6)):
    train_FLN_zero_dummy_df = pd.concat([train_FLN_zero_dummy_df, train_FLN_zero_dummy_df],axis=1)

for _ in tqdm(range(30)):
    train_FLN_zero_dummy_df = pd.concat([train_FLN_zero_dummy_df, FLN_zero_series],axis=1)

100%|██████████| 30/30 [00:39<00:00,  1.33s/it]


In [129]:
train_FLN_zero_dummy_df.columns = ['FLN_'+str(col) for col in in_test_not_in_train]
train_FLN_zero_dummy_df.tail()

,FLN_92.0,FLN_186.0,FLN_187.0,FLN_480.0,FLN_489.0,FLN_546.0,FLN_573.0,FLN_646.0,FLN_678.0,FLN_743.0,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
%%time 
train_all_FLN_dummy_df = pd.concat([train_FLN_dummy_df, train_FLN_zero_dummy_df], axis=1)

CPU times: user 57.1 s, sys: 4.75 s, total: 1min 1s
Wall time: 1min 2s


In [131]:
train_all_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2-4-2. Test FinelineNumber

In [132]:
len(test['FinelineNumber'].sort_values().unique())

5203

In [133]:
test_FLN_dummy_df = pd.get_dummies(test['FinelineNumber'], prefix='FLN_')
test_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN__9967.0,FLN__9969.0,FLN__9970.0,FLN__9971.0,FLN__9974.0,FLN__9975.0,FLN__9991.0,FLN__9997.0,FLN__9998.0,FLN__9999.0
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
%%time
# in Train, not in Test
in_train_not_in_test = \
[i for i in train_unique_fln if i not in test_unique_fln]

CPU times: user 38 ms, sys: 2.23 ms, total: 40.2 ms
Wall time: 38.3 ms


In [135]:
len(in_train_not_in_test)

150

In [136]:
FLN_zero_series = pd.Series([0]*len(test))
test_FLN_zero_dummy_df = pd.concat([FLN_zero_series, FLN_zero_series],axis=1)
for _ in tqdm(range(6)):
    test_FLN_zero_dummy_df = pd.concat([test_FLN_zero_dummy_df, test_FLN_zero_dummy_df],axis=1)

for _ in tqdm(range(22)):
    test_FLN_zero_dummy_df = pd.concat([test_FLN_zero_dummy_df, FLN_zero_series],axis=1)

100%|██████████| 22/22 [00:29<00:00,  1.34s/it]


In [137]:
test_FLN_zero_dummy_df.columns = ['FLN_'+str(col) for col in in_train_not_in_test]
test_FLN_zero_dummy_df.tail()

,FLN_45.0,FLN_238.0,FLN_245.0,FLN_332.0,FLN_373.0,FLN_524.0,FLN_551.0,FLN_567.0,FLN_593.0,FLN_830.0,...,FLN_8801.0,FLN_8903.0,FLN_8930.0,FLN_8938.0,FLN_9175.0,FLN_9527.0,FLN_9792.0,FLN_9937.0,FLN_9942.0,FLN_9964.0
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
%%time 
test_all_FLN_dummy_df = pd.concat([test_FLN_dummy_df, test_FLN_zero_dummy_df], axis=1)

CPU times: user 57.2 s, sys: 5.72 s, total: 1min 2s
Wall time: 1min 3s


In [139]:
test_all_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN_8801.0,FLN_8903.0,FLN_8930.0,FLN_8938.0,FLN_9175.0,FLN_9527.0,FLN_9792.0,FLN_9937.0,FLN_9942.0,FLN_9964.0
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

### 2-5-1. Train ScanCount

In [140]:
%%time
# in Train, not in Test
train_unique_sc = train['ScanCount'].sort_values().unique()
test_unique_sc = test['ScanCount'].sort_values().unique()

SC_in_test_not_in_train = [i for i in test_unique_sc if i not in train_unique_sc]

CPU times: user 82.4 ms, sys: 38.6 ms, total: 121 ms
Wall time: 120 ms


In [141]:
SC_in_test_not_in_train

[-21, -14, -11, -8, 26, 27, 28, 29, 35, 36, 40, 41, 42, 45, 55]

In [142]:
train_SC_dummy_df = pd.get_dummies(train['ScanCount'],prefix='SC_')

In [143]:
train_SC_dummy_df.tail()

,SC__-12,SC__-10,SC__-9,SC__-7,SC__-6,SC__-5,SC__-4,SC__-3,SC__-2,SC__-1,...,SC__20,SC__22,SC__23,SC__24,SC__25,SC__30,SC__31,SC__46,SC__51,SC__71
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
SC_zero_series = pd.Series([0]*len(train))
train_SC_zero_dummy_df = pd.concat([SC_zero_series, SC_zero_series],axis=1)
for _ in range(13):
    train_SC_zero_dummy_df = pd.concat([train_SC_zero_dummy_df, SC_zero_series],axis=1)

In [145]:
train_SC_zero_dummy_df.columns = ['SC_'+str(col) for col in SC_in_test_not_in_train]
train_SC_zero_dummy_df.tail()

,SC_-21,SC_-14,SC_-11,SC_-8,SC_26,SC_27,SC_28,SC_29,SC_35,SC_36,SC_40,SC_41,SC_42,SC_45,SC_55
647049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [146]:
%%time 
train_all_SC_dummy_df = pd.concat([train_SC_dummy_df, train_SC_zero_dummy_df], axis=1)

CPU times: user 167 ms, sys: 81 ms, total: 248 ms
Wall time: 249 ms


In [147]:
train_all_SC_dummy_df.tail()

,SC__-12,SC__-10,SC__-9,SC__-7,SC__-6,SC__-5,SC__-4,SC__-3,SC__-2,SC__-1,...,SC_27,SC_28,SC_29,SC_35,SC_36,SC_40,SC_41,SC_42,SC_45,SC_55
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2-5-2. Test ScanCount

In [148]:
%%time
# in Train, not in Test
SC_in_train_not_in_test = [i for i in train_unique_sc if i not in test_unique_sc]

CPU times: user 230 µs, sys: 1e+03 ns, total: 231 µs
Wall time: 235 µs


In [149]:
SC_in_train_not_in_test

[-12, 25, 30, 51, 71]

In [150]:
test_SC_dummy_df = pd.get_dummies(test['ScanCount'],prefix='SC_')

In [151]:
test_SC_dummy_df.tail()

,SC__-21,SC__-14,SC__-11,SC__-10,SC__-9,SC__-8,SC__-7,SC__-6,SC__-5,SC__-4,...,SC__29,SC__31,SC__35,SC__36,SC__40,SC__41,SC__42,SC__45,SC__46,SC__55
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:
SC_zero_series = pd.Series([0]*len(test))
test_SC_zero_dummy_df = pd.concat([SC_zero_series, SC_zero_series],axis=1)
for _ in range(3):
    test_SC_zero_dummy_df = pd.concat([test_SC_zero_dummy_df, SC_zero_series],axis=1)

In [153]:
test_SC_zero_dummy_df.columns = ['SC_'+str(col) for col in SC_in_train_not_in_test]
test_SC_zero_dummy_df.tail()

,SC_-12,SC_25,SC_30,SC_51,SC_71
653641,0,0,0,0,0
653642,0,0,0,0,0
653643,0,0,0,0,0
653644,0,0,0,0,0
653645,0,0,0,0,0


In [154]:
%%time 
test_all_SC_dummy_df = pd.concat([test_SC_dummy_df, test_SC_zero_dummy_df], axis=1)

CPU times: user 157 ms, sys: 39.7 ms, total: 196 ms
Wall time: 195 ms


In [155]:
test_all_SC_dummy_df.tail()

,SC__-21,SC__-14,SC__-11,SC__-10,SC__-9,SC__-8,SC__-7,SC__-6,SC__-5,SC__-4,...,SC__41,SC__42,SC__45,SC__46,SC__55,SC_-12,SC_25,SC_30,SC_51,SC_71
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

## Preprocessing Step3 - Concat VisitNumber & all Dummy DFs

### Train

- VisitNumber&파생변수 concat Weekday

In [194]:
train_df = train.loc[:,(['VisitNumber']+list(train.columns[7:]))]

In [195]:
%time train_concated_df = pd.concat([train_df, train_WD_dummy_df],axis=1)

CPU times: user 30 ms, sys: 23.3 ms, total: 53.3 ms
Wall time: 58.1 ms


- concat ScanCount

In [197]:
%time train_concated_df = pd.concat([train_concated_df,train_all_SC_dummy_df],axis=1)

CPU times: user 112 ms, sys: 97.7 ms, total: 210 ms
Wall time: 216 ms


- concat DepartmentDescription

In [198]:
%time train_concated_df = pd.concat([train_concated_df,train_DD_dummy_df],axis=1)

CPU times: user 273 ms, sys: 123 ms, total: 396 ms
Wall time: 405 ms


- concat FinelineNumber

In [199]:
%time train_concated_df = pd.concat([train_concated_df,train_all_FLN_dummy_df],axis=1)

CPU times: user 2.23 s, sys: 2.75 s, total: 4.98 s
Wall time: 5.27 s


In [200]:
train_concated_df.shape

(647054, 5494)

In [219]:
train_concated_df.tail()

,VisitNumber,Min,Max,Mean,Max - Min,FLN_count,Upc_count,DD_count,Sum,FLN_DD_Ratio,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
647049,191346,1,5,2.125,4,16,17,8,17,2.0,...,0,0,0,0,0,0,0,0,0,0
647050,191346,1,5,2.125,4,16,17,8,17,2.0,...,0,0,0,0,0,0,0,0,0,0
647051,191346,1,5,2.125,4,16,17,8,17,2.0,...,0,0,0,0,0,0,0,0,0,0
647052,191347,1,1,1.000,0,2,2,2,2,1.0,...,0,0,0,0,0,0,0,0,0,0
647053,191347,1,1,1.000,0,2,2,2,2,1.0,...,0,0,0,0,0,0,0,0,0,0


- groupby VisitNumber

In [220]:
%time final_train = train_concated_df.groupby('VisitNumber').sum()

CPU times: user 2min 2s, sys: 1min 28s, total: 3min 30s
Wall time: 3min 50s


In [221]:
final_train.shape

(95674, 5493)

In [222]:
%time train_groupbyed_TripType = train.groupby('VisitNumber')['TripType'].agg(lambda x: scipy.stats.mode(x)[0])

CPU times: user 6.36 s, sys: 70.1 ms, total: 6.43 s
Wall time: 6.5 s


In [223]:
len(train_groupbyed_TripType)

95674

In [224]:
%time final_train = pd.concat([train_groupbyed_TripType,final_train],axis=1)

CPU times: user 412 ms, sys: 323 ms, total: 734 ms
Wall time: 733 ms


In [225]:
final_train.tail()

,TripType,Min,Max,Mean,Max - Min,FLN_count,Upc_count,DD_count,Sum,FLN_DD_Ratio,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
VisitNumber,,,,,,,,,,,,,,,,,,,,,
191343,25,7,42,21.000,35,35,49,21,63,11.666667,...,0,0,0,0,0,0,0,0,0,0
191344,22,5,20,12.500,15,15,25,10,25,7.500000,...,0,0,0,0,0,0,0,0,0,0
191345,39,13,65,27.625,52,156,169,104,221,19.500000,...,0,0,0,0,0,0,0,0,0,0
191346,39,17,85,36.125,68,272,289,136,289,34.000000,...,0,0,0,0,0,0,0,0,0,0
191347,8,2,2,2.000,0,4,4,4,4,2.000000,...,0,0,0,0,0,0,0,0,0,0


In [226]:
# %time final_train.to_csv('./data/final_train_without_upc.csv')

In [227]:
%time pickle.dump(final_train, open("./data/final_train_Ver3.pkl", "wb"))

CPU times: user 395 ms, sys: 562 ms, total: 957 ms
Wall time: 1.09 s


### test

- VisitNumber&파생변수 concat Weekday

In [205]:
test_df = test.loc[:,(['VisitNumber']+list(test.columns[6:]))]

In [207]:
%time test_concated_df = pd.concat([test_df,test_WD_dummy_df],axis=1)

CPU times: user 23.2 ms, sys: 16.1 ms, total: 39.3 ms
Wall time: 39.8 ms


- concat ScanCount

In [208]:
%time test_concated_df = pd.concat([test_concated_df,test_all_SC_dummy_df],axis=1)

CPU times: user 114 ms, sys: 143 ms, total: 256 ms
Wall time: 261 ms


- concat DepartmentDescription

In [209]:
%time test_concated_df = pd.concat([test_concated_df,test_DD_dummy_df],axis=1)

CPU times: user 147 ms, sys: 164 ms, total: 310 ms
Wall time: 319 ms


- concat FinelineNumber

In [210]:
%time test_concated_df = pd.concat([test_concated_df,test_all_FLN_dummy_df],axis=1)

CPU times: user 2.35 s, sys: 4.24 s, total: 6.59 s
Wall time: 7.69 s


In [211]:
test_concated_df.shape

(653646, 5494)

- groupby VisitNumber

In [212]:
%time final_test = test_concated_df.groupby('VisitNumber').sum()

CPU times: user 2min 1s, sys: 1min 34s, total: 3min 36s
Wall time: 5min 45s


In [213]:
%time pickle.dump(final_test, open("./data/final_test_Ver3.pkl", "wb"))

CPU times: user 413 ms, sys: 585 ms, total: 998 ms
Wall time: 1.12 s


In [214]:
final_test.shape

(95674, 5493)

In [215]:
final_test.tail()

,Min,Max,Mean,Max - Min,FLN_count,Upc_count,DD_count,Sum,FLN_DD_Ratio,UPc_DD_Ratio,...,FLN_8801.0,FLN_8903.0,FLN_8930.0,FLN_8938.0,FLN_9175.0,FLN_9527.0,FLN_9792.0,FLN_9937.0,FLN_9942.0,FLN_9964.0
VisitNumber,,,,,,,,,,,,,,,,,,,,,
191338,7,63,28.000000,56,42,49,21,84,14.000000,16.333333,...,0,0,0,0,0,0,0,0,0,0
191339,5,15,10.000000,10,25,25,15,30,8.333333,8.333333,...,0,0,0,0,0,0,0,0,0,0
191340,2,2,2.000000,0,1,1,1,2,1.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
191341,10,50,24.000000,40,100,100,50,120,20.000000,20.000000,...,0,0,0,0,0,0,0,0,0,0
191348,14,21,16.333333,7,49,49,21,49,16.333333,16.333333,...,0,0,0,0,0,0,0,0,0,0


In [216]:
# %time final_test.to_csv('./data/final_test_without_upc.csv')

In [217]:
%time pickle.dump(final_test, open("./data/final_test_Ver3.pkl", "wb"))

CPU times: user 382 ms, sys: 584 ms, total: 967 ms
Wall time: 1.08 s


In [228]:
myslack_incomming.send_slack('work finish')

<Response [200]>


---